In [1]:
import requests
import re
import pandas as pd
import random
from string import ascii_lowercase
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# Preprocessing

In [2]:
# Download NLTK stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to clean text: remove punctuation, stop words, and non-textual elements
def clean_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Split into words
    words = text.split()
    # Remove stop words and non-textual elements
    cleaned_words = [word for word in words if word.lower() not in stop_words]
    # strip underscores
    cleaned_words = [word.strip('_') for word in cleaned_words]
    # remove numbers
    cleaned_words = [word for word in cleaned_words if not word.isnumeric()]
    # remove words that start with numbers
    cleaned_words = [word for word in cleaned_words if not word[0].isnumeric()]
    return ' '.join(cleaned_words)

def find_start_end(text):
    # Find the start and end of the main text
    start_pattern = r"\*\*\* START OF THIS PROJECT GUTENBERG EBOOK .+ \*\*\*"
    end_pattern = r"\*\*\* END OF THIS PROJECT GUTENBERG EBOOK .+ \*\*\*"

    start_match = re.search(start_pattern, text)
    end_match = re.search(end_pattern, text)

    start_idx = start_match.end() if start_match else 0
    end_idx = end_match.start() if end_match else len(text)

    return text[start_idx:end_idx]

def process_book(url, label):
    # Download the book text from the URL
    response = requests.get(url)
    response.encoding = 'utf-8'
    text = response.text

    # Extract the main text between start and end markers
    main_text = find_start_end(text)

    # Clean the main text
    cleaned_text = clean_text(main_text)

    # Extract words from the cleaned text
    words = cleaned_text.split()

    # Split words into partitions of 100 and take 200 random partitions
    partitions = [words[i:i + 100] for i in range(0, len(words), 100)]
    random_partitions = random.sample(partitions, min(200, len(partitions)))

    return [(label, ' '.join(partition)) for partition in random_partitions]

# Updated list of Gutenberg book URLs (Same as before, no change needed here)
book_urls = [
    'https://www.gutenberg.org/files/1342/1342-0.txt',  # Pride and Prejudice by Jane Austen
    'https://www.gutenberg.org/files/768/768-0.txt',    # Wuthering Heights by Emily Brontë
    'https://www.gutenberg.org/files/1260/1260-0.txt',  # Jane Eyre by Charlotte Brontë
    'https://www.gutenberg.org/files/1400/1400-0.txt',  # Great Expectations by Charles Dickens
    'https://www.gutenberg.org/files/145/145-0.txt',    # Middlemarch by George Eliot
    'https://www.gutenberg.org/files/541/541.txt'       # The Age of Innocence by Edith Wharton
]

book_authors = ["Jane Austen", "Emily Bronte", "Charlotte Bronte", "Charles Dickens",  "George Eliot", "Edith Wharton"]


# Process all books
all_partitions = []

for url, label in zip(book_urls, book_authors):
    book_partitions = process_book(url, label)
    all_partitions.extend(book_partitions)

# Convert to DataFrame
partition_df = pd.DataFrame(all_partitions, columns=['Label', 'Words']).sample(frac=1)

# Serialize DataFrame to CSV
partition_df.to_csv('book_partitions_cleaned.csv', index=False)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Bag of Words

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer()

In [7]:
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)
from sklearn.multiclass import OneVsRestClassifier

In [8]:
model_objs = [
    OneVsRestClassifier(SVC(kernel="rbf")),
    RandomForestClassifier(),
    OneVsRestClassifier(GaussianNB()),
    KNeighborsClassifier(),
    OneVsRestClassifier(SGDClassifier()),
    DecisionTreeClassifier(),
    OneVsRestClassifier(AdaBoostClassifier()),
    OneVsRestClassifier(XGBClassifier(random_state=69))
    ]


model_names = [
    "Gaussian SVC",
    "RandomForestClassifier",
    "Naive Bayes",
    "KNeighborsClassifier",
    "SGDClassifier",
    "DecisionTreeClassifier",
    "AdaBoostClassifier",
    "XGBClassifier",
]

In [10]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

kf = KFold(n_splits=10, shuffle=True, random_state=69)
onehot_encoder = OneHotEncoder(sparse=False)

for m_obj, m_name in zip(model_objs, model_names):
  acc_sum = 0
  precision_sum = 0
  recall_sum = 0
  f1_sum = 0
  roc_auc_sum = 0
  for train_index,test_index in kf.split(partition_df[:600]):
    train_data = partition_df.iloc[train_index]
    test_data = partition_df.iloc[test_index]
    X_train = train_data.drop(['Label'], axis=1)
    y_train = train_data['Label']
    X_test = test_data.drop(['Label'], axis=1)
    y_test = test_data['Label']
    # One-hot encoding for the label
    y_train_onehot = onehot_encoder.fit_transform(train_data[['Label']])
    y_test_onehot = onehot_encoder.transform(test_data[['Label']])

    y_train_df = pd.DataFrame(y_train_onehot, columns=onehot_encoder.get_feature_names_out(['Label']))
    y_test_df = pd.DataFrame(y_test_onehot, columns=onehot_encoder.get_feature_names_out(['Label']))

    X_train_bow = bow_vectorizer.fit_transform(X_train['Words'])
    X_test_bow = bow_vectorizer.transform(X_test['Words'])
    X_train_bow = pd.DataFrame(X_train_bow.toarray(), columns=bow_vectorizer.get_feature_names_out())
    X_test_bow = pd.DataFrame(X_test_bow.toarray(), columns=bow_vectorizer.get_feature_names_out())

    m_obj.fit(X_train_bow, y_train_df),
    y_pred = m_obj.predict(X_test_bow)
    acc = accuracy_score(y_pred, y_test_df)
    precision = precision_score(y_pred, y_test_df, average='macro')
    recall = recall_score(y_pred, y_test_df, average='macro')
    f1 = f1_score(y_pred, y_test_df, average='macro')
    roc_auc = roc_auc_score(y_test_df, y_pred, average='macro')

    acc_sum+=acc
    precision_sum += precision
    recall_sum += recall
    f1_sum += f1
    roc_auc_sum += roc_auc

  print("MODEL: {}".format(m_name))
  print("Average Accuracy: {}".format(acc_sum/10))
  print("Average Precision: {}".format(precision_sum/10))
  print("Average Recall: {}".format(recall_sum/10))
  print("Average F1: {}".format(f1_sum/10))
  print("Average ROC AUC: {}".format(roc_auc_sum/10))
  print()


# Plotting the graph
metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']

plt.figure(figsize=(10, 8))
plt.subplots_adjust(bottom=0.3)

for i, metric_list in enumerate([accuracy_list, precision_list, recall_list, f1_list, roc_auc_list]):
    plt.plot(model_names, metric_list, label=metrics[i])


plt.title('Model Performance Comparison')
plt.xlabel('Models')

# Rotate model names vertically
plt.xticks(rotation=90, ha='center')

plt.ylabel('Metric Value')
plt.legend()
plt.show()

MODEL: Gaussian SVC
Average Accuracy: 0.4066666666666666
Average Precision: 0.3911202454952455
Average Recall: 0.8166666666666667
Average F1: 0.5029425516029367
Average ROC AUC: 0.6955601227476228

MODEL: RandomForestClassifier
Average Accuracy: 0.45333333333333325
Average Precision: 0.4379909211159211
Average Recall: 0.9166666666666667
Average F1: 0.5656918687308712
Average ROC AUC: 0.7189954605579605

MODEL: Naive Bayes
Average Accuracy: 0.28833333333333333
Average Precision: 0.28225980038480036
Average Recall: 0.8305555555555554
Average F1: 0.3998623817702765
Average ROC AUC: 0.6409487407721104

MODEL: KNeighborsClassifier
Average Accuracy: 0.545
Average Precision: 0.5322489316239316
Average Recall: 0.8258295329043411
Average F1: 0.5678368777996274
Average ROC AUC: 0.7454802348856148

MODEL: SGDClassifier
Average Accuracy: 0.7449999999999999
Average Precision: 0.7523293835793836
Average Recall: 0.9312614237614237
Average F1: 0.8216665953896738
Average ROC AUC: 0.8710208203196448

MO

# TF IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [12]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

kf = KFold(n_splits=10, shuffle=True, random_state=69)
onehot_encoder = OneHotEncoder(sparse=False)

for m_obj, m_name in zip(model_objs, model_names):
  acc_sum = 0
  precision_sum = 0
  recall_sum = 0
  f1_sum = 0
  roc_auc_sum = 0
  for train_index,test_index in kf.split(partition_df[:600]):
    train_data = partition_df.iloc[train_index]
    test_data = partition_df.iloc[test_index]
    X_train = train_data.drop(['Label'], axis=1)
    y_train = train_data['Label']
    X_test = test_data.drop(['Label'], axis=1)
    y_test = test_data['Label']
    # One-hot encoding for the label
    y_train_onehot = onehot_encoder.fit_transform(train_data[['Label']])
    y_test_onehot = onehot_encoder.transform(test_data[['Label']])

    y_train_df = pd.DataFrame(y_train_onehot, columns=onehot_encoder.get_feature_names_out(['Label']))
    y_test_df = pd.DataFrame(y_test_onehot, columns=onehot_encoder.get_feature_names_out(['Label']))

    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['Words'])
    X_test_tfidf = tfidf_vectorizer.transform(X_test['Words'])
    X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
    X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

    m_obj.fit(X_train_tfidf, y_train_df),
    y_pred = m_obj.predict(X_test_tfidf)
    acc = accuracy_score(y_pred, y_test_df)
    precision = precision_score(y_pred, y_test_df, average='macro')
    recall = recall_score(y_pred, y_test_df, average='macro')
    f1 = f1_score(y_pred, y_test_df, average='macro')
    roc_auc = roc_auc_score(y_test_df, y_pred, average='macro')

    acc_sum+=acc
    precision_sum += precision
    recall_sum += recall
    f1_sum += f1
    roc_auc_sum += roc_auc

  print("MODEL: {}".format(m_name))
  print("Average Accuracy: {}".format(acc_sum/10))
  print("Average Precision: {}".format(precision_sum/10))
  print("Average Recall: {}".format(recall_sum/10))
  print("Average F1: {}".format(f1_sum/10))
  print("Average ROC AUC: {}".format(roc_auc_sum/10))
  print()

  # Plotting the graph
metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']

plt.figure(figsize=(10, 8))
plt.subplots_adjust(bottom=0.3)

for i, metric_list in enumerate([accuracy_list, precision_list, recall_list, f1_list, roc_auc_list]):
    plt.plot(model_names, metric_list, label=metrics[i])


plt.title('Model Performance Comparison')
plt.xlabel('Models')

# Rotate model names vertically
plt.xticks(rotation=90, ha='center')

plt.ylabel('Metric Value')
plt.legend()
plt.show()

KeyboardInterrupt: 

# Word embedding

In [13]:
glove_file_path= 'glove.6B/glove.6B.50d.txt'

# Function to load GloVe embeddings into a dictionary
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Load GloVe embeddings into the dictionary
glove_embeddings = load_glove_embeddings(glove_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B/glove.6B.50d.txt'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Assuming you have defined 'model' and 'names' somewhere in your code

kf = KFold(n_splits=10, shuffle=True, random_state=69)
onehot_encoder = OneHotEncoder(sparse=False, drop='first')

# Lists to store metric values for each model
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
roc_auc_list = []

for m_obj, m_name in zip(model_objs, model_names):
    acc_sum = 0
    precision_sum = 0
    recall_sum = 0
    f1_sum = 0
    roc_auc_sum = 0

    for train_index, test_index in kf.split(partition_df):
        train_data = partition_df.iloc[train_index]
        test_data = partition_df.iloc[test_index]
        X_train = train_data['text']
        y_train = train_data['label']
        X_test = test_data['text']
        y_test = train_data['label']

        # Create document embeddings using GloVe
        X_train_glove = np.array([np.mean([glove_embeddings.get(word, np.zeros(50)) for word in sentence.split()], axis=0) for sentence in X_train])
        X_test_glove = np.array([np.mean([glove_embeddings.get(word, np.zeros(50)) for word in sentence.split()], axis=0) for sentence in X_test])

        # One-hot encoding for the label
        y_train_onehot = onehot_encoder.fit_transform(train_data[['label']])
        y_test_onehot = onehot_encoder.transform(test_data[['label']])
        y_train_df = pd.DataFrame(y_train_onehot, columns=onehot_encoder.get_feature_names_out(['label']))
        y_test_df = pd.DataFrame(y_test_onehot, columns=onehot_encoder.get_feature_names_out(['label']))

        #Model Training and Prediction
        m_obj.fit(X_train_glove, y_train_df)
        y_pred = m_obj.predict(X_test_glove)

        # Compute Evaluation Metrics
        acc = accuracy_score(y_pred, y_test_df)
        precision = precision_score(y_pred, y_test_df, average='macro')
        recall = recall_score(y_pred, y_test_df, average='macro')
        f1 = f1_score(y_pred, y_test_df, average='macro')
        roc_auc = roc_auc_score(y_test_df, y_pred, average='macro')

        acc_sum += acc
        precision_sum += precision
        recall_sum += recall
        f1_sum += f1
        roc_auc_sum += roc_auc

    # Calculate average metric values for the model
    avg_acc = acc_sum / 10
    avg_precision = precision_sum / 10
    avg_recall = recall_sum / 10
    avg_f1 = f1_sum / 10
    avg_roc_auc = roc_auc_sum / 10

    # Append the average metric values to the respective lists
    accuracy_list.append(avg_acc)
    precision_list.append(avg_precision)
    recall_list.append(avg_recall)
    f1_list.append(avg_f1)
    roc_auc_list.append(avg_roc_auc)

    print("MODEL: {}".format(m_name))
    print("Average Accuracy: {}".format(avg_acc))
    print("Average Precision: {}".format(avg_precision))
    print("Average Recall: {}".format(avg_recall))
    print("Average F1: {}".format(avg_f1))
    print("Average ROC AUC: {}".format(avg_roc_auc))
    print()

# Plotting the graph
metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']

plt.figure(figsize=(10, 8))
plt.subplots_adjust(bottom=0.3)

for i, metric_list in enumerate([accuracy_list, precision_list, recall_list, f1_list, roc_auc_list]):
    plt.plot(model_names, metric_list, label=metrics[i])


plt.title('Model Performance Comparison')
plt.xlabel('Models')

# Rotate model names vertically
plt.xticks(rotation=90, ha='center')

plt.ylabel('Metric Value')
plt.legend()
plt.show()

# BERT

In [ ]:
!pip install tensorflow_text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

In [ ]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dense(128, activation='relu', name='hidden_1')(net)
  net = tf.keras.layers.Dropout(0.3)(net)
  net = tf.keras.layers.Dense(6, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(bert_raw_result)

In [ ]:
tf.keras.utils.plot_model(classifier_model)

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = tf.metrics.CategoricalAccuracy()

In [ ]:
X = partition_df.drop(['Label'], axis=1)
y = partition_df['Label']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import warnings
import numpy as np
warnings.filterwarnings('ignore')

onehot_encoder = OneHotEncoder(sparse=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_onehot = onehot_encoder.fit_transform(np.array(y_train).reshape(-1,1))
y_test_onehot = onehot_encoder.transform(np.array(y_test).reshape(-1,1))

y_train_df = pd.DataFrame(y_train_onehot, columns=onehot_encoder.get_feature_names_out(['Label']))
y_test_df = pd.DataFrame(y_test_onehot, columns=onehot_encoder.get_feature_names_out(['Label']))

In [ ]:
y_train_df

In [ ]:
epochs = 10
optimizer = tf.keras.optimizers.Adam(0.0001)

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
classifier_model.summary()

In [ ]:
history = classifier_model.fit(X_train,y_train_df, batch_size=128, epochs=epochs, validation_data=(X_test, y_test_df))

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_pred = to_categorical(np.argmax(classifier_model.predict(X_test), axis=-1))

In [ ]:
y_pred

In [ ]:
acc = accuracy_score(y_pred, y_test_df)
precision = precision_score(y_pred, y_test_onehot, average='macro')
recall = recall_score(y_pred, y_test_onehot, average='macro')
f1 = f1_score(y_pred, y_test_onehot, average='macro')
roc_auc = roc_auc_score(y_test_onehot, y_pred, average='macro')

In [ ]:
print("Average Accuracy: {}".format(acc))
print("Average Precision: {}".format(precision))
print("Average Recall: {}".format(recall))
print("Average F1: {}".format(f1))
print("Average ROC AUC: {}".format(roc_auc))

In [ ]:
classifier_model.save('BERT_classifier.h5')

# Error analysis: BERT